In [8]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score

In [9]:
data = pd.read_csv('f1_cleaned.csv')
data = data.rename(columns={'driver_name' : 'team_name'})
data.dtypes

team_name         object
code              object
driver_nat        object
circuitRef        object
year             float64
round            float64
starting_pos     float64
finishing_pos    float64
laps             float64
quali_mean       float64
driver_age       float64
driver_dnf         int64
car_dnf            int64
dtype: object

In [10]:
#train test split
#we are not using a random split here, training with pre 2024 data and trying to predict the races that occured in 2024

train = data[data.year<2024]
test = data[data.year==2024]

#testing set
y_test = test.pop('finishing_pos')
x_test = test

#training set
y_train = train.pop('finishing_pos')
x_train = train


In [11]:
#encoding vars and scaling data

cat_feat = ['team_name', 'code', 'driver_nat', 'circuitRef']
x_num_feat = ['year', 'round', 'starting_pos', 'laps', 'quali_mean', 'driver_age', 'driver_dnf', 'car_dnf']

#scale y later if needed for a distance model

ct = ColumnTransformer(transformers=[
    ('encoder', OneHotEncoder(handle_unknown='ignore', drop='first'), cat_feat), #avoid dummy var trap with OHE
    ('scx', StandardScaler(), x_num_feat)
])




In [ ]:
#Block for parameter tuning

In [ ]:
#Basic LR model (no tuning)

model = Pipeline(steps=[
    ('preprocessor', ct),
    ('regressor', LinearRegression())
])

model.fit(x_train, y_train)
y_pred = model.predict(x_test)

r2 = r2_score(y_test, y_pred)
print(r2)

0.6229827414899718


/opt/anaconda3/envs/ML/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [1, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [13]:
#df created to compare results 

comparison_df = pd.DataFrame({
    'Driver': test['code'],              
    'Circut': test['circuitRef'],          
    'Actual Pos': y_test,                         
    'Predicted Pos': y_pred                       
})